File: ratings.ipynb\
Author: alexkobz\
Date: 07.05.2024\
Task: DSRFU-275

#### Шаг 0. Импорт и получение токена

In [1]:
import asyncio
import requests
import pandas as pd
import numpy as np
import nest_asyncio; nest_asyncio.apply()
from datetime import datetime as dt, date, timedelta
from functions.divide_chunks import divide_chunks
from math import ceil
from functions.get_date import last_day_month, last_day_month_str, last_work_date_month, last_work_date_month_str

# последний день месяца
REPORT_DATE = last_day_month
REPORT_DATE_STR = last_day_month_str
# увеличиваем количество столбцов и строк для просмотра
pd.set_option('display.max_rows', 200, 'display.max_columns', 200)

In [2]:
from rudata.RuDataDF import RuDataDF
from rudata.RuDataRequest import RuDataRequest
RuDataRequest.set_headers()

Getting token started
Getting token finished


#### Шаг 1. Загрузка stage таблиц 

[FintoolReferenceData](https://docs.efir-net.ru/dh2/#/Info/FintoolReferenceData?id=post-fintoolreferencedata) - Получить расширенный справочник по финансовым инструментам.

In [3]:
FintoolReferenceData = RuDataDF("FintoolReferenceData").df
FintoolReferenceData.head()

FintoolReferenceData started
FintoolReferenceData finished. FintoolReferenceData shape (102423, 186)


,facevalue_m,faceftname_m,sumissuevol_m,summarketvol_m,numcoupons_m,nrdcode,regcode_m,regdate_m,regdistdate_m,begdistdate_m,enddistdate_m,endmtydate_m,daysall_m,fullname_nrd,rpregorg_nrd,rpstate_nrd,issuenumber,currentfacevalue_nrd,isqualified_nrd,bondseries,islombardcbr_nrd,numcoupons_nrd,facevalue_nrd,faceftname_nrd,summarketvol_nrd,regcode_nrd,regdate_nrd,regorg_nrd,begdistdate_nrd,daysall_nrd,enddistdate_nrd,endmtydate_nrd,regdistdate_nrd,sumissuevol_nrd,drqty,isincodebase_nrd,nrdcompcode,shqty,summarketvolorig_nrd,issuername_nrd,planmtydate_nrd,firstcoupondate_nrd,depositary_nrd,registrar_nrd,placeprice_nrd,micexlist_nrd,minlot_nrd,formarketbonds_nrd,registraracctype_nrd,registraracctypedate_nrd,facialacc_nrd,acc_open_date_nrd,fraction_nrd,specialized_depo_nrd,cmp_code_nsd_issuer_nrd,cmp_code_nsd_managing_nrd,ratetypenamerus_nrd,ratetypenameeng_nrd,coupontypename_nrd,base_month,base_year,basis_nrd,sectypenamerus_nrd,sectypenameeng_nrd,secformnamerus_nrd,secformnameeng_nrd,sectypenamebr_nrd,secstaterus_nrd,secstateeng_nrd,placementtype_nrd,share_category_name_nrd,share_category_name_en_nrd,dr_category_name_nrd,dr_category_name_en_nrd,company_state_name_nrd,rate_type_id,cp_type_id,coupon_period_base_id,sec_type_id,sec_form_id,sec_state_id,placement_type_id,share_category_id,dr_category_mn,company_state_mn,sector4212u,sna2008,fullname_en_nrd,seniorityname,fintoolid,moex_code,moex_name,fullname,isin144a,isincode,nickname,progfintoolid,regcode,regdate,regorg,amortisedmty,basis,bondstructuralpar,cfi,coupontype,fintooltype,guaranteeamount,guaranteetype,haveoffer,isconvertible,isguaranteed,maturitygroup,securitization,accruedintcalctype,country,couponperyear,faceftname,facevalue,guarantval,havedefault,numcoupons,status,sumissueval,sumissuevol,summarketval,summarketvol,borrowername,borrowerokpo,borrowersector,borroweruid,borrowerinn,borrowercountry,borrowerregionname,issuername,issuerokpo,issuersector,issueruid,issuerinn,issuercountry,issuerregionname,assetssector,numguarantors,begdistdate,begmtydate,daysall,enddistdate,endmtydate,privatedist,regdistdate,securitytype,securitykind,nominaltype,update_time,isregion,nsd_service,raterevisionsperyear,floatratename,issubordinated,note,listing_level,havecovenant,haverepayment,haveindexedfv,numdecimals,couponperiod,ismatched,firstcoupondate,couponrate,iscomplicated,available_for_unqualified,sec_type_br_code,original_country,qualified_test_id,qualified_test_id_tm,base_asset_fintoolid,isqualified,listing_level_spb,have_mm_moex,have_mm_spb,beg_book_date,seniority,tradesite,current_coupontype,profit_type,counter,rn
0,2.0,2.0,NaN,NaN,NaN,None,0.0,0.0,0.0,0.0,0.0,NaN,NaN,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,209185,None,None,"Ventoux CCM Acquisition Corp, Ordinary Shares",None,US92280L1017,Ventoux CCM -US92280L1017,NaN,None,None,None,NaN,None,None,None,None,Акция,None,None,NaN,0.0,NaN,None,None,NaN,US,NaN,USD,0.0001,NaN,NaN,NaN,В обращении,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,None,Ventoux CCM Acquisition,None,None,120453.0,None,US,None,None,NaN,None,None,NaN,None,None,0.0,None,Обыкн-ая,Неопределен,None,2023-10-12T00:25:07,NaN,Нет информации,NaN,None,NaN,None,NaN,None,None,None,NaN,NaN,None,None,NaN,None,False,None,US,1000,1000,NaN,None,NaN,False,False,None,NaN,102.0,None,None,102423,82863
1,2.0,2.0,NaN,NaN,NaN,None,0.0,0.0,0.0,0.0,0.0,NaN,NaN,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,N

[SecurityRatingTable](https://docs.efir-net.ru/dh2/#/Rating/SecurityRatingTable?id=post-securityratingtable) - Получить рейтинги нескольких бумаг и связанных с ними компаний на заданную дату.

In [ ]:
%%time
SecurityRatingTable = RuDataDF("SecurityRatingTable").df
SecurityRatingTable.head()

SecurityRatingTable started


[ExchangeTree](https://docs.efir-net.ru/dh2/#/Info/ExchangeTree?id=post-exchangetree) - Получить иерархию торговых площадок/источников, используемых Интерфакс

In [ ]:
ExchangeTree = RuDataDF("ExchangeTree").df
ExchangeTree.head()

[ListScaleValues](https://docs.efir-net.ru/dh2/#/Rating/ListScaleValues?id=post-listscalevalues) - Список шкал значений рейтингов

In [ ]:
ListScaleValues = RuDataDF("ListScaleValues").df
ListScaleValues.head()

[ListRatings](https://docs.efir-net.ru/dh2/#/Dictionary/ListRatings?id=post-listratings) - Список рейтингов

In [ ]:
ListRatings = RuDataDF("ListRatings").df
ListRatings.head()

[Emitents](https://docs.efir-net.ru/dh2/#/Info/Emitents?id=post-emitents) - Получить краткий справочник по эмитентам.

In [ ]:
Emitents = RuDataDF("Emitents").df
Emitents.head()

[OfferorsGuarants](https://docs.efir-net.ru/dh2/#/Bond/OfferorsGuarants?id=post-offerorsguarants) - Возвращает список гарантов/оферентов для инструмента

In [ ]:
OfferorsGuarants = RuDataDF("OfferorsGuarants").df
OfferorsGuarants.head()

[CompanyRatingsTable](https://docs.efir-net.ru/dh2/#/Rating/CompanyRatingsTable?id=post-companyratingstable) - Получить рейтинги нескольких компаний на заданную дату.

In [ ]:
CompanyRatingsTable = RuDataDF("CompanyRatingsTable").df
CompanyRatingsTable.head()

#### Шаг 2. Предобработка таблиц до экспорта в excel

In [ ]:
#добавляем информацию из справочника в нашу таблицу с компаниями
ExtendedSecurityRatingTable = SecurityRatingTable.merge(ListRatings[['rating_id','agency','official_name','term_type_name','scale_type_name','scale_id']], how='left', left_on='id_rating', right_on='rating_id')
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable.merge(ListScaleValues, how='left', left_on=['scale_id', 'id_value'], right_on=['scale_id', 'rating_value'])
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[[
    'id_x', 'hid', 'code_name', 'id_value', 'forecast', 'change',
    'date_award', 'role', 'okpo', 'fininstid', 'fintoolid', 'isin',
    'id_rating', 'code', 'shortname_rus', 'rating_id', 'agency',
    'official_name', 'term_type_name', 'scale_type_name', 'scale_id',
    'aggregation_level', 'aggregation_type']].rename(
    columns={
    'id_x': 'id', 'aggregation_level': 'level', 'aggregation_type': 'agg_type'
    }
)
agencies = ['АКРА', 'Эксперт РА', 'НКР', 'НРА'
           # ]
            , 'Мудис Интерфакс']
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[ExtendedSecurityRatingTable['agency'].isin(agencies)]
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[ExtendedSecurityRatingTable['term_type_name'] != 'Краткосрочный']
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[~ExtendedSecurityRatingTable['id_value'].isin(['****','Приостановлен','Снят'])]
ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[~ExtendedSecurityRatingTable['isin'].isna()]


In [ ]:
akra_list = [
'Structured Finance Instrument Rating',
 'Рейтинг эмиссии, международная шкала',
 'Старший необеспеченный рейтинг эмиссии',
 'Старший обеспеченный рейтинг эмиссии',
 'Субординированный рейтинг эмиссии'
]
expert_list = [
    'Structured finance instrument rating',
    'Кредитный рейтинг долговых инструментов'
]
nkr_list = [
   # 'Кредитный рейтинг долговых обязательств'
]
nra_list = [
 #   'Кредитный рейтинг отдельных выпусков облигаций',
  #  'Рейтинг кредитоспособности выпусков облигаций Индивидуальный'
]
moodys_list = [
    'NSR Senior Unsecured (Domestic)',
 'Senior Unsecured (Domestic); Long-Term Corporate Obligation Rating (Domestic); Structured Finance Long-Term Rating (Domestic)',
 'Senior Unsecured (Foreign); Long-Term Corporate Obligation Rating (Foreign); Structured Finance Long-Term Rating (Foreign)',
 'Subordinate (Domestic)',
 'Subordinate (Foreign)'
]

ExtendedSecurityRatingTable = ExtendedSecurityRatingTable[
((ExtendedSecurityRatingTable['agency']=='АКРА') & (ExtendedSecurityRatingTable['official_name'].isin(akra_list))) |
((ExtendedSecurityRatingTable['agency']=='Эксперт РА') & (ExtendedSecurityRatingTable['official_name'].isin(expert_list))) |
((ExtendedSecurityRatingTable['agency']=='НКР') & (ExtendedSecurityRatingTable['official_name'].isin(nkr_list))) |
((ExtendedSecurityRatingTable['agency']=='НРА') & (ExtendedSecurityRatingTable['official_name'].isin(nra_list))) |
((ExtendedSecurityRatingTable['agency']=='Мудис Интерфакс') & (ExtendedSecurityRatingTable['official_name'].isin(moodys_list)))
 ]

ExtendedSecurityRatingTable = ExtendedSecurityRatingTable\
.dropna(subset=['isin'])\
.drop_duplicates(subset=["isin", "agency"])\
.reset_index(drop=True)

In [ ]:
ExtendedSecurityRatingTablePivot = ExtendedSecurityRatingTable.drop_duplicates().pivot(index='isin', columns='agency', values='id_value')
ExtendedSecurityRatingTablePivot.columns = pd.MultiIndex.from_tuples([('Эмиссия', i) for i in ExtendedSecurityRatingTablePivot.columns])

In [ ]:
TransformedFintoolReferenceData = FintoolReferenceData.copy()[['isincode', 'nickname', 'fintooltype', 'country', 'faceftname', 'fintoolid',
                   'issuername', 'issuercountry', 'issuerinn', 'issueruid',
                   'borrowername', 'borrowercountry', 'borrowerinn', 'borroweruid',
                   'isguaranteed', 'guaranteetype', 'guaranteeamount', 'guarantval',
                   ]].drop_duplicates(subset=['isincode'])
TransformedFintoolReferenceData[['fintoolid', 'issueruid', 'borroweruid']] = TransformedFintoolReferenceData[['fintoolid', 'issueruid', 'borroweruid']].fillna(0).astype('int')
TransformedFintoolReferenceData = TransformedFintoolReferenceData.merge(OfferorsGuarants[['fintoolId','fininstid_guarant']], how='left', left_on='fintoolid', right_on='fintoolId')
ids = list(TransformedFintoolReferenceData.loc[~TransformedFintoolReferenceData['fininstid_guarant'].isna(),'fininstid_guarant'].astype('int'))
TransformedEmitents = Emitents[Emitents.fininstid.isin(ids)][['shortname_rus','country','inn','fininstid']]
TransformedEmitents.columns = ['name_guarant','country_guarant','inn_guarant','fininstid']
TransformedFintoolReferenceData = TransformedFintoolReferenceData.merge(TransformedEmitents, how='left', left_on='fininstid_guarant', right_on='fininstid')
TransformedFintoolReferenceData = TransformedFintoolReferenceData.drop(columns=['fintoolId', 'fininstid'])
TransformedFintoolReferenceData = TransformedFintoolReferenceData.set_index('isincode').rename_axis('ISIN')
TransformedFintoolReferenceData = TransformedFintoolReferenceData.rename(
    columns={
        'nickname': ('Эмиссия','Название'),
        'fintooltype': ('Эмиссия','Тип'),
        'country': ('Эмиссия','Страна'), 
        'faceftname': ('Эмиссия','Валюта'), 
        'fintoolid': ('Эмиссия','Id RuData'),
        'issuername': ('Эмитент','Название'), 
        'issuercountry': ('Эмитент','Страна'), 
        'issuerinn': ('Эмитент','ИНН'), 
        'issueruid': ('Эмитент','Id RuData'), 
        'borrowername': ('Заемщик','Название'),
        'borrowercountry': ('Заемщик','Страна'), 
        'borrowerinn': ('Заемщик','ИНН'), 
        'borroweruid': ('Заемщик','Id RuData'), 
        'isguaranteed': ('Гарант','Есть гарантия'),
        'guaranteetype': ('Гарант','Вид гарантии'), 
        'guaranteeamount': ('Гарант','Полнота гарантии'), 
        'guarantval': ('Гарант','Гарантированная сумма'), 
        'fininstid_guarant': ('Гарант','Id RuData'),
        'name_guarant': ('Гарант','Название'), 
        'country_guarant': ('Гарант','Страна'), 
        'inn_guarant': ('Гарант','ИНН')
    }
)
TransformedFintoolReferenceData.columns = pd.MultiIndex.from_tuples(TransformedFintoolReferenceData.columns)

In [ ]:
ExtendedFintoolReferenceData = TransformedFintoolReferenceData.join(ExtendedSecurityRatingTablePivot)
new_cols = ExtendedFintoolReferenceData.columns.reindex(['Эмиссия','Эмитент','Заемщик','Гарант'], level=0)
ExtendedFintoolReferenceData = ExtendedFintoolReferenceData.reindex(columns=new_cols[0])

In [ ]:
ExtendedCompanyRatingsTable = CompanyRatingsTable.merge(ListRatings[['id','agency','official_name','term_type_name','scale_type_name','scale_id']], how='left', left_on='id_rating', right_on='id')
##добавляем уровень рейтинг
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.merge(ListScaleValues, how='left', left_on=['scale_id', 'last'], right_on=['scale_id', 'rating_value'])

In [ ]:
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[['name_ra', 'dt', 'last', 'last_dt', 'change', 'forecast', 'advanced',
           'prev', 'prev_dt', 'id_rating', 'code', 'code_type', 'company_name',
           'id_x', 'agency', 'official_name', 'term_type_name', 'scale_type_name',
           'scale_id', 'aggregation_level', 'aggregation_type']].rename(columns={'id_x': 'id', 'aggregation_level': 'level', 'aggregation_type': 'agg_type'})
ExtendedCompanyRatingsTable['id_rating'] = ExtendedCompanyRatingsTable['id_rating'].fillna(0)

In [ ]:
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[ExtendedCompanyRatingsTable['agency'].isin(['АКРА','Эксперт РА','НКР','НРА','Мудис Интерфакс'])]
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[~ExtendedCompanyRatingsTable['last'].isin(['****','Приостановлен','Снят'])]
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[ExtendedCompanyRatingsTable['term_type_name'] != 'Краткосрочный']

In [ ]:
akra_list=[
 'Рейтинг эмитентов',
 'Long-term credit rating',
 'Long-term local currency credit rating',
 'Long-term foreign currency credit rating',
]
expert_list=[
 'Рейтинг кредитоспособности организаций, осуществляющих микрофинансовую деятельность (Республика Казахстан)',
 'Рейтинг кредитоспособности проектных компаний',
 'Рейтинги кредитоспособности банка Республики Беларусь',
 'Рейтинги кредитоспособности банков',
 'Рейтинги кредитоспособности депозитариев',
 'Рейтинги кредитоспособности лизинговых компаний',
 'Рейтинги кредитоспособности микрофинансовых организаций',
 'Рейтинги кредитоспособности нефинансовых компаний',
 'Рейтинги кредитоспособности нефинансовых компаний Республики Беларусь',
 'Рейтинги кредитоспособности нефинансовых компаний Республики Казахстан',
 'Рейтинги кредитоспособности субъектов федерации и муниципальных образований',
 'Рейтинги кредитоспособности факторинговых компаний',
 'Рейтинги кредитоспособности финансовых компаний',
 'Рейтинги кредитоспособности холдинговых компаний',
 'Рейтинги надежности компаний по страхованию жизни',
 'Рейтинги финансовой надежности негосударственных пенсионных фондов',
 'Рейтинги финансовой надежности страховых компаний'
]
nkr_list=[
 'Кредитный рейтинг НКР по Национальной шкале, Банки',
 'Кредитный рейтинг НКР по Национальной шкале, Нефинансовые компании'
]
nra_list=[
 'Кредитный рейтинг кредитных организаций',
 'Кредитный рейтинг нефинансовых компаний, national scale',
]
moodys_list=[
 'BACKED LT Bank Deposits',
 'BACKED LT Bank Deposits (Domestic)',
 'BACKED LT Bank Deposits (Foreign)',
 'BACKED LT Counterparty Risk Rating',
 'BACKED LT Counterparty Risk Rating (Domestic)',
 'BACKED LT Counterparty Risk Rating (Foreign)',
 'BACKED LT Issuer Rating',
 'BACKED LT Issuer Rating (Domestic)',
 'BACKED LT Issuer Rating (Foreign)',
 'BACKED Senior Unsecured (Domestic)',
 'BACKED Senior Unsecured (Foreign)',
 'Issuer Ratings',
 'LT Bank Deposits',
 'LT Bank Deposits (Domestic)',
 'LT Bank Deposits (Foreign)',
 'LT Bank Deposits (kaz)',
 'LT Bank Deposits (ukr)',
 'LT Corporate Family Ratings',
 'LT Corporate Family Ratings (Domestic)',
 'LT Corporate Family Ratings (Foreign)',
 'LT Corporate Family Ratings (ukr)',
 'LT Counterparty Risk Rating (Domestic)',
 'LT Counterparty Risk Rating (Foreign)',
 'LT Insurance Financial Strength',
 'LT Insurance Financial Strength (Domestic)',
 'LT Insurance Financial Strength (Foreign)',
 'LT Issuer Rating',
 'LT Issuer Rating (Domestic)',
 'LT Issuer Rating (Foreign)',
 'Senior Unsecured',
 'Senior Unsecured (Domestic)',
 'Senior Unsecured (Foreign)',
]

ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable[
((ExtendedCompanyRatingsTable['agency']=='АКРА') & (ExtendedCompanyRatingsTable['official_name'].isin(akra_list))) |
((ExtendedCompanyRatingsTable['agency']=='Эксперт РА') & (ExtendedCompanyRatingsTable['official_name'].isin(expert_list))) |
((ExtendedCompanyRatingsTable['agency']=='НКР') & (ExtendedCompanyRatingsTable['official_name'].isin(nkr_list))) |
((ExtendedCompanyRatingsTable['agency']=='НРА') & (ExtendedCompanyRatingsTable['official_name'].isin(nra_list))) |
((ExtendedCompanyRatingsTable['agency']=='Мудис Интерфакс') & (ExtendedCompanyRatingsTable['official_name'].isin(moodys_list)))
]

In [ ]:
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.dropna(subset=['code'])
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.reset_index(drop=True)

In [ ]:
for n, g in ExtendedCompanyRatingsTable.groupby(['code', 'agency']):
    if g.shape[0] > 1: 
        try:
            m = g['level'].idxmin(skipna=False)
            if pd.isna(m):
                m = g.index[0]
            to_drop = g.index.drop(m)
            ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.drop(index=to_drop)
        except:
            print(g)
            raise
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.dropna(subset=['code'])
ExtendedCompanyRatingsTable = ExtendedCompanyRatingsTable.reset_index(drop=True)
ExtendedCompanyRatingsTable['code'] = ExtendedCompanyRatingsTable['code'].astype(int)

In [ ]:
ExtendedCompanyRatingsPivot = ExtendedCompanyRatingsTable.pivot(index='code', columns='agency', values='last')

In [ ]:
df1 = ExtendedFintoolReferenceData.copy()
df1.columns =df1.columns.to_flat_index()
for k in ['Эмитент','Заемщик','Гарант']:
    df2 = ExtendedCompanyRatingsPivot.copy()
    df2.columns = [(k, i) for i in df2.columns]
    df1 = df1.merge(df2, how='left', left_on=[(k, 'Id RuData')], right_index=True)
df1.columns = pd.MultiIndex.from_tuples(df1.columns)
new_cols = df1.columns.reindex(['Эмиссия', 'Эмитент', 'Заемщик', 'Гарант'], level=0)
df1 = df1.reindex(columns=new_cols[0])
ExtendedFintoolReferenceData = df1

In [ ]:
df = Emitents[Emitents['fininstid'].isin(ExtendedCompanyRatingsPivot.index)]
CompanyRatings = df.merge(ExtendedCompanyRatingsPivot, left_on='fininstid', right_index=True, how='left')

#### Шаг 3. Рейтинги облигаций из фактовой таблицы бумаг FintoolReferenceData. Рейтинги облигаций из фактовой таблицы компаний CompanyRatingsTable

In [ ]:
BondRatings = ExtendedFintoolReferenceData['Эмиссия'].reset_index()
id_vars = BondRatings.columns[:6]
value_vars = BondRatings.columns[6:]
meltBonds = pd.melt(BondRatings, id_vars=id_vars, value_vars=value_vars, var_name='agency', value_name='rate')
meltBonds = meltBonds[~meltBonds.rate.isna()].sort_values('Id RuData')
meltBonds = meltBonds[meltBonds["Тип"] == 'Облигация']

In [ ]:
CompanyRatings = pd.concat([
    CompanyRatings[["id_emitent", "shortname_rus", "inn", "update_date", "country_oksm", "have_rating", "have_risk", "tin", "fininstid", "reg_code", "lei_code"]]
    , CompanyRatings.iloc[:, CompanyRatings.columns.get_loc('rn')+1:]
], axis=1)
id_vars = CompanyRatings.columns[:11]
value_vars = CompanyRatings.columns[11:]
meltCompanies = pd.melt(CompanyRatings, id_vars=id_vars, value_vars=value_vars, var_name='agency', value_name='rate')
meltCompanies = meltCompanies[meltCompanies['rate'].notnull()].sort_values('id_emitent')
meltCompanies['inn'] = np.where(meltCompanies['country_oksm'] == 643, meltCompanies['inn'], meltCompanies['tin'] )
meltCompanies['country_oksm'] = meltCompanies['country_oksm'].astype(np.int32)
meltCompanies['type'] = "ЮЛ"

In [ ]:
meltBonds.rename(columns={
    "Id RuData": "id_emitent",
    "Название": "company_name", 
    "Тип": "rate_type",
    "ISIN": "inn/isin",
    "Страна": "country"
    }, inplace=True)
meltCompanies.rename(columns={
    "id_rudata": "id_emitent",
    "shortname_rus": "company_name", 
    "type": "rate_type",
    "inn": "inn/isin",
    "country_oksm": "country_code"
    }, inplace=True)
meltBonds["fininstid"] = np.nan
meltBonds["reg_code"] = np.nan
meltBonds["lei_code"] = np.nan
meltBonds["report_date"] = pd.to_datetime(REPORT_DATE)
meltCompanies["report_date"] = pd.to_datetime(REPORT_DATE)

Справочник кодов стран ОКСМ

In [ ]:
countries = pd.read_sql(
    """
    SELECT DISTINCT country, alpha2
    FROM countries
    """
    , RuDataDF.engine)
countries["country"] = countries["country"].astype(np.int32)
co = countries.set_index("alpha2").country.to_dict()
meltBonds["country_code"] = meltBonds["country"].map(co)

#### Шаг 4. Получаем результирующий результат по рейтингам облигаций и компаний и экспорт в excel

In [ ]:
meltBonds = meltBonds[["id_emitent", "fininstid", "company_name", "inn/isin", "country_code", "agency", "rate", "report_date", "rate_type", "reg_code", "lei_code"]]
meltCompanies = meltCompanies[["id_emitent", "fininstid", "company_name", "inn/isin", "country_code", "agency", "rate", "report_date", "rate_type", "reg_code", "lei_code"]]
ResultRatings = pd.concat([meltBonds, meltCompanies], axis=0)
with pd.ExcelWriter('data/Output/ratings.xlsx', datetime_format='dd.mm.yyyy') as writer:
    ResultRatings.to_excel(writer, index=False)

In [ ]:
from functions.send_email import main
main("ratings.xlsx")